In [136]:
from math import log
import operator

**Decision Tree implementation**

In [137]:
#Calculate shannon entropy
def calcShannonEnt(dataSet):
    numEntries = len(dataSet)
    labelCounts = {}
    for featureVec in dataSet:
        currentLabel = featureVec[-1]
        if currentLabel not in labelCounts.keys():
            labelCounts[currentLabel] = 0
        labelCounts[currentLabel] += 1
    shannonEnt = 0.0
    for key in labelCounts:
        prob = float(labelCounts[key])/numEntries
        shannonEnt -= prob * log(prob,2)
    return shannonEnt

In [138]:
#Generate dataset for testing
def createDataSet():
    dataSet = [[1, 1, 'yes'],
               [1, 1, 'yes'],
               [1, 0, 'no'],
               [0, 1, 'no'],
               [0, 1, 'no']]
    labels = ['no surfacing', 'flippers']
    return dataSet, labels

In [139]:
def splitDataSet(dataSet, axis, value):
    retDataSet=[]
    for featureVec in dataSet:
        if featureVec[axis] == value:
            reducedFeatureVec = featureVec[:axis]
            reducedFeatureVec.extend(featureVec[axis+1:])
            retDataSet.append(reducedFeatureVec)
    return retDataSet

In [140]:
def chooseBestFeatureToSplit(dataSet):
    numFeatures = len(dataSet[0]) - 1
    #We need this because information gain need this base entropy to substract
    baseEntropy = calcShannonEnt(dataSet)
    bestInfoGain = 0.0; bestFeature = -1
    for i in range(numFeatures):
        featureList = [example[i] for example in dataSet]
        uniqueVals = set(featureList)
        newEntropy = 0.0
        for value in uniqueVals:
            subDataSet = splitDataSet(dataSet, i, value)
            prob = len(subDataSet) / float(len(dataSet))
            newEntropy += prob * calcShannonEnt(subDataSet)
        infoGain = baseEntropy - newEntropy
        if(infoGain > bestInfoGain):
            bestInfoGain = infoGain
            bestFeature = i
    return bestFeature

In [141]:
"""
Return the category with most frequency
"""
def majorityCnt(classList):
    classCount = {}
    for vote in classCount:
        if vote not in classCount.keys():
            classCount[vote] = 0
        classCount[vote] += 1
        sortedClassCount = sorted(classCount.items(), key=operator.itemgetter(1), reverse=True)
    return sortedClassCount[0][0]

In [142]:
def createTree(dataSet, labels):
    #Get all labels
    classList = [entry[-1] for entry in dataSet]
    #If categories are the same, stop creating tree
    if classList.count(classList[0]) == len(classList):
        return classList[0]
    #Return label with most frequency
    if len(dataSet[0]) == 1:
        return majorityCnt(classList)
    #Pick best feature using previous algorithm
    bestFeature = chooseBestFeatureToSplit(dataSet)
    bestFeatureLabel = labels[bestFeature]
    myTree = {bestFeatureLabel: {}}
    del(labels[bestFeature])
    featureValues = [entry[bestFeature] for entry in dataSet]
    #Get cardinality
    uniqueVals = set(featureValues)
    for value in uniqueVals:
        #Copy label
        subLabels = labels[:]
        #Recursively call createTree()
        myTree[bestFeatureLabel][value] = createTree(splitDataSet(dataSet, bestFeature, value), subLabels)
    return myTree

In [143]:
dataSet, label = createDataSet()
dataSet_origin, label_origin = createDataSet()
myTree= createTree(dataSet, label)
myTree

{'no surfacing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}}

***Example***

In [144]:
import numpy as np

In [145]:
def classify(inputTree,featLabels,testVec):
    firstStr = list(inputTree.keys())[0]
    secondDict = inputTree[firstStr]
    featIndex = featLabels.index(firstStr)
    key = testVec[featIndex]
    valueOfFeat = secondDict[key]
    if isinstance(valueOfFeat, dict): 
        classLabel = classify(valueOfFeat, featLabels, testVec)
    else: classLabel = valueOfFeat
    return classLabel

In [164]:
for i in range(10):
    firstFeat = random.randint(0,2)
    secondFeat = random.randint(0,2)
    input = np.array([firstFeat, secondFeat])
    prediction = classifyDT(myTree, label_origin, input)
    print('Decision tree predicts that ', input, 'belongs to', prediction)

Decision tree predicts that  [1 0] belongs to no
Decision tree predicts that  [1 0] belongs to no
Decision tree predicts that  [0 0] belongs to no
Decision tree predicts that  [1 1] belongs to yes
Decision tree predicts that  [1 1] belongs to yes
Decision tree predicts that  [0 0] belongs to no
Decision tree predicts that  [1 1] belongs to yes
Decision tree predicts that  [1 0] belongs to no
Decision tree predicts that  [0 1] belongs to no
Decision tree predicts that  [1 1] belongs to yes
